In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")

In [3]:
cur = con.execute("select * from sqlite_master")
cur.fetchall()

[]

In [4]:
inserts = []
inserts.append(["moje1", 1, 1.5])
inserts.append(["moje2", 2, 2.5])
inserts.append(["moje3", 3, None])
inserts

[['moje1', 1, 1.5], ['moje2', 2, 2.5], ['moje3', 3, None]]

In [5]:
cur = con.execute("create table aaa (id integer primary key autoincrement, col1 varchar, col2 int, col3 float)")
con.commit()

In [6]:
cur = con.execute("select * from sqlite_master")
cur.fetchall()

[('table',
  'aaa',
  'aaa',
  2,
  'CREATE TABLE aaa (id integer primary key autoincrement, col1 varchar, col2 int, col3 float)'),
 ('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)')]

In [7]:
cur.executemany("insert into aaa values(null, ?, ?, ?)", inserts)
con.commit()

In [8]:
cur = con.execute("select * from aaa")
cur.fetchall()

[(1, 'moje1', 1, 1.5), (2, 'moje2', 2, 2.5), (3, 'moje3', 3, None)]

In [9]:
# cur = conn.cursor(cursor_factory = psycopg2.extras.NamedTupleCursor)
from collections import namedtuple
def namedtuple_factory(cursor, row):
    """
    Usage:
    con.row_factory = namedtuple_factory
    """
    fields = [col[0] for col in cursor.description]
    Row = namedtuple("Row", fields)
    return Row(*row)
con.row_factory = namedtuple_factory

In [10]:
cur = con.execute("select * from aaa")
cur.fetchall()

[Row(id=1, col1='moje1', col2=1, col3=1.5),
 Row(id=2, col1='moje2', col2=2, col3=2.5),
 Row(id=3, col1='moje3', col2=3, col3=None)]

In [11]:
cur = con.execute("select * from aaa")
data = cur.fetchone()
while data:
    print (data.id)
    data = cur.fetchone()

1
2
3


In [12]:
cur = con.execute("select * from aaa")
df = pd.DataFrame(cur.fetchall())
df

,id,col1,col2,col3
0,1,moje1,1,1.5
1,2,moje2,2,2.5
2,3,moje3,3,NaN


In [22]:
cur = con.execute('''select *, sum(col2) over() sumcol1, sum(col2) over(order by id) sumcol2 from aaa;''')
df = pd.DataFrame(cur.fetchall())
df

,id,col1,col2,col3,sumcol1,sumcol2
0,1,moje1,1,1.5,6,1
1,2,moje2,2,2.5,6,3
2,3,moje3,3,NaN,6,6
